In [1]:
import tensorflow as tf
import os
import sys
import math
import numpy

In [ ]:
# Apply Add-Lambda smoothing function on language model.
"""Load language model

Load language model from folder "lm" and save them into dictionary "lm".

Args:
    loadPath: language model load path.
    encoding: language model files' encoding

Returns:
    lm: a tensorflow HashTable of language model. Its structure is:
    
    {"unigram": unigram,
     "bigram" : bigram,
     "trigram": trigram}.
"""
def loadLM( loadPath = "./lm", encoding = "utf-8" ):
    tlm = {}
    ngram = ["unigram", "bigram", "trigram"]
    # load unigram, bigram, and trigram
    for name in ngram:
        with open( loadPath + "/" + name, "r", encoding = encoding ) as f:
            ngram = {}
            total = 0
            line = f.readline()
            while line:
                kv = line.split( ' ' )
                if len( kv ) > 1:
                    k = ' '.join( kv[:-1] )
                    v = kv[-1]
                    ngram[k] = int( v )
                else:
                    pass
                line = f.readline()
            tlm[name] = tf.HashTable( tf.KeyValueTensorInitializer(
                                        ngram.keys(), ngram.values() ), 0 )
    lm = tf.HashTable( tf.KeyValueTensorInitializer( tlm.keys(), tlm.values() ), 0 )
    return lm

"""Add lambda smoothing

P(w_{n}|w_{n-1}w_{n-2}) = ( c(w_{n-1}w_{n-2}, w_{n}) + lambda ) /
                            ( c(w_{n-1}w_{n-1}) + lambda * V )

Args:
    lm: a tensorflow HashTable of language model. Its structure is:
    
    {"unigram": unigram,
     "bigram" : bigram,
     "trigram": trigram}.
    
    lambdas: a generator of lambdas which generate a dictionary of lambdas
             for unigram, bigram, trigram each time. For example:
    
    {1:0.1, 2:0.1, 3:0.8}
    
    s: a list of words wating for calculating unigram, bigram, and trigram.

Returns:
    p: a double number represents the final probability of P(w_{n}|w_{n-2}w_{n-1}).
"""
def addLambda( lm, lambdas, s ):
    s = tf.slice( s, [-1] )
    if tf.equal( lm.lookup( "trigram" ).lookup( s ), 0 ):
        cnt1 = 0
    else:
        cnt1 = lm.lookup( "trigram" ).lookup( s )
    s = ' '.join( s[:-1] )
    if s not in lm["bigram"]["c"]:
        cnt2 = 0
    else:
        cnt2 = lm["bigram"]["c"][s]
    p = ( cnt1 + lambdas[3] ) / ( cnt2 + len( lm["bigram"]["c"] ) * lambdas[3] )
    return p

"""Calculate perplexity

PP(W) = P(w_1w_2 ... w_n)^(-1/n)
      = 2^{-1 / n * sum_{i=1:n}(log2(LM(w_i|w_{i-2}w_{i-1})))}

Note: Since here is no <SOS> and <EOS> in language model, n would be the length of
      the content - 2.

Args:
    contents: a tensor contains words in a sentences. Each line represents a sentence.
    lm: a tensorflow HashTable of language model. Its structure is:
    
    {"unigram": unigram,
     "bigram" : bigram,
     "trigram": trigram}.

    **kwargs:
        func: smoothing function name on calculating P(w_i|w_{i-2}w_{i-1}), including
              func = "Interplotation" and func = "AddLambda".
        
        lambdas: a dictionary of lambda for interplotation or addLambda. Its structure is:
    
        {1: lambdaForUnigram, 2:lambdaForBigram, 3:lambdaForTrigram}
        
        When using addLambda function, only need to feed one specific lambda.
    
Returns:
    ppw: a double number represents the perplexity of the content.

Raise:
    KeyError: an error when trying to find smoothing function.
"""
def perplexity( contents, lm, **kwargs ):
    numOfSentence = contents.shape[0]
    length = contents.shape[1]
    logP = tf.Variable( 0, name = "logP" )
    if( length <= 2 ):
        raise Exception( "Too short content." )
    if "func" in kwargs:
        if kwargs["func"] == "AddLambda":
            for i in range( numOfSentence ):
                content = tf.strided_clise( contents, [i], [i + 1] )
                for i in range( length - 2 ):
                    p = tf.Variable( 0, name = "p" )
                    p = addLambda( lm, kwargs["lambdas"], tf.strided_slice( content, [:, i], [:, i + 3] ) )
                    logP = logP + tf.math.log( p )
        else:
            raise Exception( "Cannot find the smoothing function." )
    else:
        raise Exception( "No smoothing function." )
    logP = logP * -1 / ( length - 2 )
    ppw = tf.math.exp( logP )
    return ppw

In [33]:
a = tf.Variable( [[1, 2], [3, 4]] )
c = tf.strided_slice( a, [0], [1] )

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run( init )
    print( sess.run( a ) )
    for i in range( a.shape[0] ):
        print( sess.run( tf.strided_slice( a, [i], [i + 1] ) ) )

[[1 2]
 [3 4]]
[[1 2]]
[[3 4]]
